In [2]:
import json

import h5py
import numpy as np
from glove import Glove

def load_geovec(path):
    instance = Glove("")
    with h5py.File(path, 'r') as f:
        v = np.zeros(f['vectors'].shape, f['vectors'].dtype)
        f['vectors'].read_direct(v)
        dct = f['dct'][()].tostring().decode('utf-8')
        dct = json.loads(dct)
    instance.word_vectors = v
    instance.no_components = v.shape[1]
    instance.word_biases = np.zeros(v.shape[0])
    instance.add_dictionary(dct)
    return instance

new_model = load_geovec('geovec_300d_v1.h5')

/home/jgosses82/miniconda3/envs/geovec/lib/python3.6/site-packages/ipykernel_launcher.py:12: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  if sys.path[0] == '':


AttributeError: 'Glove' object has no attribute 'add_dictionary'

In [2]:
!ls

geovec_300d_v1.h5  README.md  Untitled.ipynb


In [4]:
def load_geovec(path):
    instance = Glove("")
    with h5py.File(path, 'r') as f:
        v = np.zeros(f['vectors'].shape, f['vectors'].dtype)
        f['vectors'].read_direct(v)
        dct = f['dct'][()].tostring().decode('utf-8')
        dct = json.loads(dct)
    return v,dct

In [5]:
v,dct = load_geovec('geovec_300d_v1.h5')

/home/jgosses82/miniconda3/envs/geovec/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  


In [6]:
v

array([[ 0.6575543 , -0.10153764, -0.2825202 , ...,  0.64620304,
         0.3083481 , -0.46784   ],
       [ 0.06665467,  0.11548886,  0.22080564, ..., -0.08355215,
         0.00378662, -0.13758856],
       [ 0.32676426,  0.26151726, -0.33332917, ...,  0.15112405,
         0.00892717, -0.19385871],
       ...,
       [ 0.02525451, -0.00096976,  0.04692964, ..., -0.05422716,
        -0.02048381,  0.01153759],
       [ 0.09515277,  0.09372371,  0.09677262, ..., -0.10224963,
        -0.06350729, -0.05045411],
       [-0.02760863,  0.0040131 ,  0.03059224, ..., -0.04461955,
        -0.03546656, -0.00330813]], dtype=float32)

In [9]:
len(v[0])

300

In [7]:
dct

{'acrodermatitis': 391621,
 'groenlandica': 78252,
 'microbaculatus': 177591,
 'typotheriopsis': 109964,
 'cvesicle': 222066,
 'epss': 39171,
 'mników': 342521,
 'istin': 115124,
 'vlvg': 287983,
 'heatflux': 348681,
 'rhizodeposit': 115220,
 'lipped': 158884,
 'springflood': 112469,
 'orbicula': 192526,
 'tallied': 19354,
 'heberti': 234526,
 'cesspool': 87168,
 'windbreaker': 261688,
 'bassian': 204364,
 'groslier': 342626,
 'overbeek': 137882,
 'californica': 66315,
 'vanuato': 355064,
 'pfsg': 400053,
 'overwhelmed': 18730,
 'amt': 23680,
 'hellenikon': 128267,
 'syria': 13072,
 'supratemporal': 69061,
 'valdoviño': 306967,
 'lithogenic': 15416,
 'freiburg': 40002,
 'martonne': 79876,
 'biezelingse': 378947,
 'eccl': 365511,
 'glissant': 382948,
 'perreau': 352496,
 'microtherm': 371072,
 'strobed': 274457,
 'jaupart': 59975,
 'tetralones': 408428,
 'lugh': 201485,
 'palaeoaquifers': 396988,
 'bakanasensis': 231219,
 'alestinae': 245085,
 'afon': 95245,
 'eastalpine': 261056,
 'sul